In [66]:
%matplotlib inline
# notebook
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import pandas as pd
import networkx as nx
from math import sin, cos, sqrt, atan2, radians
from utils import *
from data import *
from trophics import *
from collections import defaultdict
#make the graphs bigger
pylab.rcParams['figure.figsize'] = (32.0, 24.0)
pylab.rcParams['font.size'] = 24

In [67]:
eiggSet = validatedEiggData()
eiggCommon = eiggSet['Common name']
eiggCommon = set(eiggCommon)

eiggSet = eiggSet['Scientific name']
eiggSet = set(eiggSet)

len(eiggSet)

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2982

In [40]:
def defineNewRowHeadersJaneData(df):
    firstNameRow = df.iloc[0].values.tolist()
    secondNameRow = df.iloc[1].values.tolist()
    combinesToTuples = list(zip(firstNameRow,secondNameRow))
    combinesToTuples = list(filter(lambda x: (type(x[0]) is str) and (type(x[1]) is str), combinesToTuples ))
    combinedToSingleNameRow = list(map(lambda x: (x[0].lower()+" "+x[1].lower()).strip(), combinesToTuples))

    return combinedToSingleNameRow

def defineNewColHeadersJaneData(df):
    namedCols = list(zip(df.iloc[:,0].values.tolist(),df.iloc[:,1].values.tolist()))
    namedCols = namedCols[4:]
    namedCols = list(map(lambda x: (x[0],'') if type(x[1]) is not str else x, namedCols))
    combinedToSingleNameCol = list(map(lambda x: (x[0].lower()+" "+x[1].lower()).strip(), namedCols))

    return combinedToSingleNameCol

In [75]:
df = pd.read_csv('./RelevantDatasets/InsightPending/plantPollinator.csv')
dfPollinators = df['POLLINATOR_NAME']
dfPlants = df['PLANT_NAME']

pollinators = list(set(dfPollinators.values.tolist()))
pollinators = list(map(lambda x: cleanEcologicalName(x),pollinators))

plants = list(set(dfPlants.values.tolist()))
plants = list(map(lambda x: cleanEcologicalName(x),plants))

totals = set()

for item in plants:
    if item in eiggSet or item in eiggCommon:
        totals.add(item)
        
for item in pollinators:
    if item in eiggSet or item in eiggCommon:
        totals.add(item)

len(totals)

263

In [65]:
df = pd.read_csv('./RelevantDatasets/InsightPending/floridaNames')
df = df['name']
df = list(map(lambda x: cleanEcologicalName(x),df))
df = list(set(df))
for item in df:
    if item in eiggSet or item in eiggCommon:
        print(item)

In [63]:
df = pd.read_csv('./RelevantDatasets/InsightPending/Otago/Otago_Data_Nodes.csv')
df = df['WorkingName'].values.tolist()
df = set(df)
df = list(map(lambda x: cleanEcologicalName(x),df))
for item in df:
    if item in eiggSet or item in eiggCommon:
        print(item)

In [50]:
df = pd.read_excel('./RelevantDatasets/InsightPending/Otago/plantPollinatorUSA.xls',header=None)
cols = defineNewRowHeadersJaneData(df)
cols = list(map(lambda x: cleanEcologicalName(x), cols))
cols

['plant_ge plant_sp',
 'claytonia virginica',
 'dentaria laciniata',
 'dicentra canadensis',
 'dicentra cucullaria',
 'erythronium albidum',
 'isopyrum biternatum',
 'sanguinaria canadensis']

In [51]:
rows = defineNewColHeadersJaneData(df)
rows = list(map(lambda x: cleanEcologicalName(x), rows))
(rows)

['andrena cressoni',
 'andrena irgeniae',
 'andrena erythronii',
 'andrena forbesii',
 'andrena mansonii',
 'andrena miserabilis',
 'andrena personata',
 'andrena rugosa',
 'andrena',
 'ceratina metallica',
 'nomada luteola',
 'bombus greseocollis',
 'augochlora pura',
 'augochlorella striata',
 'dialictus coeruleus',
 'dialictus oblongus',
 'dialictus obscurus',
 'dialictus imitatus',
 'dialictus zephyrus',
 'erylaeus macroupinensis',
 'halictus confusus',
 'lasioglossum forbesii',
 'carposcalis obscura',
 'eristalis dimidiatus',
 'helophilus fasciatus',
 'melanostoma',
 'mesograpta marginata',
 'metasyrphus americanus',
 'surphus torvus',
 'syrphus',
 'xylota nemorum']

In [52]:
for item in cols:
    if item in eiggSet:
        print(item)

In [53]:
for item in rows:
    if item in eiggSet:
        print(item)